In [1]:
import pathlib
from fytok.Tokamak import Tokamak
from fytok.modules.Equilibrium import Equilibrium
from fytok.utils.logger import logger
from spdm.view.View import display
WORKSPACE = "/home/salmon/workspace"  # "/ssd01/salmon_work/workspace/"
output_path = pathlib.Path(f"{WORKSPACE}/output/")

2023-12-07 08:19:15,405 [    spdm]     INFO: 
###################################################################################################

    ______      _____     _
   / ____/_  __|_   _|__ | | __
  / /_  / / / /  | |/ _ \| |/ /
 / __/ / /_/ /   | | (_) |   <
/_/    \__, /    |_|\___/|_|\_\
      /____/

 Copyright (c) 2021-present Zhi YU (Institute of Plasma Physics Chinese Academy of Sciences)
            
 url: https://gitee.com/openfusion/fytok_tutorial 
      https://github.com/fusion-yun/fytok_tutorial

 version = alpha  (spdm=beta extension=0.2.2.dev19+g14adade.d20231205, imas_version=v3_38_1_dirty)

 Run by salmon at 2023-12-07T08:19:15.021109.
 Job ID: fytok_salmon_surface_11381

###################################################################################################



In [2]:
# eq0 = Equilibrium(f"file+GEQdsk://{WORKSPACE}/gacode/neo/tools/input/profile_data/g141459.03890#equilibrium")
tok = Tokamak(
    "file+geqdsk:///home/salmon/workspace/fytok_tutorial/tutorial/data/g900003.00230_ITER_15MA_eqdsk16HR.txt",
    device="east",
    equilibrium={"code": {"name": "freegs"}},
)

2023-12-07 08:19:15,784 [    spdm]     INFO: Load module spdm.plugins.data.plugin_xml
2023-12-07 08:19:15,801 [    spdm]     INFO: Load module spdm.plugins.data.plugin_geqdsk


In [3]:
tok.refresh()

2023-12-07 08:19:16,428 [    spdm]     INFO: Load module fytok.plugins.equilibrium.freegs
2023-12-07 08:19:16,433 [    spdm]     INFO: Initialize module fytok.modules.Wall 
2023-12-07 08:19:16,440 [    spdm]     INFO: Initialize module fytok.modules.TF 
2023-12-07 08:19:16,446 [    spdm]     INFO: Initialize module fytok.modules.Magnetics 
2023-12-07 08:19:16,461 [    spdm]     INFO: Initialize module fytok.modules.PFActive 
2023-12-07 08:19:16,471 [    spdm]     INFO: Initialize module fytok.plugins.equilibrium.freegs 


KeyboardInterrupt: 

In [ ]:
profiles_2d.grid